   ## Aaconda 更新
   Open Anaconda Prompt, **run as admin**
   > conda update --all
   
   ## NLTK 安装

In [1]:
import nltk
#nltk.set_proxy('http://127.0.0.1:1080', ('USERNAME', 'PASSWORD'))
nltk.set_proxy('http://proxy:8080', ('USERNAME', 'PASSWORD'))
#nltk.download()

In [2]:
# 或者，直接指定下载哪个
# nltk.download('popular')

会弹出来一个窗口：
- Collections
- Corpora 语料
- Models 模型
- All Packages

## NLTK 自带语料库

In [3]:
# 如果之前没有下载过 brown 语料库，可以临时下载
# nltk.download('brown')

# brown大学的语料库，包含很多分类
from nltk.corpus import brown

# 如果遇到 "No Disc" error
# Navigate to: HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Windows\ 
# Change the value of the ErrorMode key to 2.

# 看看包含多少分类
print(brown.categories())

# 看看多少句子，多少单词
print(len(brown.sents()))
print(len(brown.words()))

['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies', 'humor', 'learned', 'lore', 'mystery', 'news', 'religion', 'reviews', 'romance', 'science_fiction']
57340
1161192


## 文本处理流程
- Preprocess 预处理
  - 包含 Tokenize 分词，等等其它步骤
- 生成 Features 特征 (as X)
- 机器学习

## Tokenize
把长句拆成有意义的小部件

In [4]:
import nltk
sentence = "hello, world"
tokens = nltk.word_tokenize(sentence)
tokens

['hello', ',', 'world']

## 中英文NLP区别
例句：今天/天气/不错
- 启发式 Heuristic，寻找最长的拟合词
- 机器学习/统计方法：HMM（隐马尔科夫链），CRF
  - 斯坦福的 CoreNLP 可以支持中文
  
## 中文分词
- 安装jieba:

Open Anaconda Prompt
> pip install --proxy 127.0.0.1:1080 jieba

or
> pip install jieba

也可以研究下CoreNLP，也有中文分词功能

In [5]:
import jieba
seg_list = jieba.cut("我来到北京清华大学", cut_all=True) # 全模式
"/ ".join(seg_list)

Building prefix dict from the default dictionary ...
Loading model from cache D:\Temp\jieba.cache
Loading model cost 0.934 seconds.
Prefix dict has been built succesfully.


'我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学'

In [6]:
seg_list = jieba.cut("我来到北京清华大学", cut_all=False) # 精确模式
"/ ".join(seg_list)

'我/ 来到/ 北京/ 清华大学'

In [7]:
seg_list = jieba.cut("他来到了网易杭研大厦") # 默认精确模式
"/ ".join(seg_list)
# 注意：这里“杭研”并没有在词典中，但也被Viterbi算法识别出来了

'他/ 来到/ 了/ 网易/ 杭研/ 大厦'

In [8]:
seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造") # 搜索引擎模式
"/ ".join(seg_list)
# 该模式所有可能的都排列出来了，更适合于搜索引擎的使用

'小明/ 硕士/ 毕业/ 于/ 中国/ 科学/ 学院/ 科学院/ 中国科学院/ 计算/ 计算所/ ，/ 后/ 在/ 日本/ 京都/ 大学/ 日本京都大学/ 深造'

## 有时候分词没有那么简单
例如：
- RT是转发
- @
- :) 笑脸
- 等等

In [9]:
from nltk.tokenize import word_tokenize
tweet = 'RT @angelababy: love you baby! :D http://ah.love #168cm'
print(word_tokenize(tweet))

['RT', '@', 'angelababy', ':', 'love', 'you', 'baby', '!', ':', 'D', 'http', ':', '//ah.love', '#', '168cm']


## 社交语言的 tokenize

对于下面的`emoticons_str`，例子：
- `:-)`
- `:)`

正则对照表: http://www.regexlab.com/zh/regref.htm

In [10]:
import re

emoticons_str = r"""
    (?:
        [:=;] # 眼睛
        [oO\-]? # 鼻子
        [D\)\]\(\]/\\OpP] # 嘴
    )"""

regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # at 某人
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # 话题标签
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # 数字
    r"(?:[a-z][a-z'\-_]+[a-z])", # 含有 - 和 ‘ 的单词，例如 don't
    r'(?:[\w_]+)', # 其他
    r'(?:\S)' # 其他
]

In [15]:
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

tweet = 'RT @angelababy: love you baby! :D http://ah.love #168cm'
print(preprocess(tweet))

['RT', '@angelababy', ':', 'love', 'you', 'baby', '!', ':D', 'http://ah.love', '#168cm']
